In [ ]:
import pandas as pd

1. Categorical Variables

In [ ]:
#df = pd.read_csv("IML_DataSets\Adult.csv")
df2 = pd.read_excel("DMforBA_DataSets\ToyotaCorolla.xls")
df2 = df2.loc[:, df2.columns != 'Id']
df2.head()

In [ ]:
df = df2[['Price', 'Age_08_04', 'KM', 'Fuel_Type', 'Automatic', 'Gears']]
display(df.head(10))

one hot encoding

In [ ]:
df_onehot = pd.get_dummies(df)
print("Original features:\n", list(df.columns), "\n")
print("Features after get_dummies:\n", list(df_onehot.columns))

dummy variables

In [ ]:
df_dummy = pd.get_dummies(df,drop_first=True)
print("Original features:\n", list(df.columns), "\n")
print("Features after get_dummies:\n", list(df_dummy.columns))

In [ ]:
import statsmodels.api as sm

In [ ]:
X = df_dummy.loc[:, df_dummy.columns != 'Price']
y = df_dummy[['Price']]
X = sm.add_constant(X)
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

In [ ]:
X2 = df_onehot.loc[:, df_onehot.columns != 'Price']
y2 = df_onehot[['Price']]
X2 = sm.add_constant(X2)
mod = sm.OLS(y2, X2)
res = mod.fit()
print(res.summary())

Holdout and CrossValidation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
reg2 = LinearRegression().fit(X_train, y_train)
y_pred_test = reg2.predict(X_test)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test))
print("Coefficient of determination: %.3f" % r2_score(y_test, y_pred_test))

X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size = 0.3)
reg2 = LinearRegression().fit(X_train, y_train)
y_pred_test = reg2.predict(X_test)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test))
print("Coefficient of determination: %.3f" % r2_score(y_test, y_pred_test))

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from sklearn.linear_model import LinearRegression

In [ ]:
score_dummy = []
score_onehot = []
s_no = []
for i in range(0,10):
    # prepare the cross-validation procedure
    cv = KFold(n_splits=10, random_state=i, shuffle=True)
    reg = LinearRegression()
    
    # evaluate model
    scores = cross_val_score(reg, X, y, cv=cv) 
    score_dummy.append(mean(scores))

    scores = cross_val_score(reg, X2, y2, cv=cv) 
    score_onehot.append(mean(scores))
    
    s_no.append(i)
    
scores_df = pd.DataFrame(
    {'S #': s_no,
     'dummy': score_dummy,
     'onehot': score_onehot
    })
scores_df.head(10)

In [ ]:
from sklearn.model_selection import LeaveOneOut

In [ ]:
score_dummy = []
score_onehot = []
s_no = []
for i in range(0,10):
    # prepare the cross-validation procedure
    cv = LeaveOneOut()
    reg = LinearRegression()
    
    # evaluate model
    scores = cross_val_score(reg, X, y, cv=cv) 
    score_dummy.append(mean(scores))

    scores = cross_val_score(reg, X2, y2, cv=cv) 
    score_onehot.append(mean(scores))
    
    s_no.append(i)
    
scores_df = pd.DataFrame(
    {'S #': s_no,
     'dummy': score_dummy,
     'onehot': score_onehot
    })
scores_df.head(10)

3. Filter/Wrapper

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
#cor = df.corr()
cor = df2.corr()
#sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
sns.heatmap(cor, annot=False, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["Price"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.4]
relevant_features

In [ ]:
df3 = df2[['Price', 'Age_08_04', 'Mfg_Year','KM', 'Weight','Airco', 'Automatic_airco', 
          'Boardcomputer','CD_Player']]
plt.figure(figsize=(12,10))
#cor = df.corr()
cor = df3.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
#sns.heatmap(cor, annot=False, cmap=plt.cm.Reds)
plt.show()

4. AIC, BIC and Adjusted-R2

In [ ]:
mod = sm.OLS(y, X)
res = mod.fit()
print("AIC: %.2f, BIC: %.2f, R2-Adj: %.2f" % (res.aic, res.bic, res.rsquared_adj))
#"Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test)

In [ ]:
X.dtypes

In [ ]:
X_Red2 = X[['const', 'Age_08_04', 'Gears', 'Automatic', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol']]
mod = sm.OLS(y, X_Red2)
res = mod.fit()
print("AIC: %.2f, BIC: %.2f, R2-Adj: %.2f" % (res.aic, res.bic, res.rsquared_adj))

In [ ]:
X_Red1 = X[['const', 'Age_08_04', 'KM', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol']]
mod = sm.OLS(y, X_Red1)
res = mod.fit()
print("AIC: %.2f, BIC: %.2f, R2-Adj: %.2f" % (res.aic, res.bic, res.rsquared_adj))